In [3]:
from lang import *
from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")


In [9]:
lang_conf_params = {
    "max_len" : 4096
}
longformer_lang_conf = LangConf(vocab_size=50000,**lang_conf_params)
lang = LanguageIndex(config=longformer_lang_conf)
lang.load_tokenizer(tokenizer)

In [33]:
text1 = """At least 13 people, including 12 schoolchildren, were killed when the school bus they were travelling in was involved in a head-on collision with a truck in Aliganj area of Etah district in Uttar Pradesh this morning. District administration and police officials told NetIndian that at least 15 other children had suffered injuries and were admitted to nearby hospitals for treatment. Among those killed was the driver of the bus, they said, adding that the bus was ferrying about 50 children in all to their school when the mishap occurred around 8 am. "Tragic road accident in Aliganj Etah. Over 15 school kids feared dead. Rescue of injured ongoing," Uttar Pradesh Director General of Police Javeed Ahmed said on micro-blogging site Twitter. The school had remained open today even though the Etah district administration had asked all schools to be closed for three days till January 20 because of the ongoing cold wave in the region, sources said. According to various accounts, the accident occurred at a time when visibility in the area was poor because of dense fog. Cranes were called in to lift the bus which had turned turtle because of the collision and gas cutters had to be used to cut through the mangled steel to extricate the victims from the heavily damaged vehicle.   and Prime Minister Narendra Modi condoled the loss of lives in the accident. "Heartfelt condolences on loss of young lives in tragic bus accident in Etah; prayers for speedy recovery of the injured," Mr. Mukherjee said on Twitter. "Anguished by the tragic accident in UP’s Etah district. I share the pain of the bereaved families & condole passing away of young children," Mr. Modi said. "I pray that those injured in the accident in Etah recover at the earliest," he added..  Fifteen young children were killed and twenty five others seriously injured this morning when their school bus crashed into a truck in Uttar Pradesh's Etah. The truck was speeding when the school bus, with more than 60 students, collided with it.The children on the bus were junior school students between seven and ten years old.Prime Minister Narendra Modi expressed grief in tweets. "Anguished by the tragic accident in UP's Etah district. I share the pain of the bereaved families & condole passing away of young children," tweeted the Prime Minister. Anguished by the tragic accident in UP's Etah district. I share the pain of the bereaved families & condole passing away of young children. - Narendra Modi (@narendramodi) January 19, 2017 - Narendra Modi (@narendramodi) January 19, 2017 The police have confirmed that all the children have been rescued from the bus, and the injured have been taken to the district hospital. 16 children, who have serious injuries, have been shifted to a specialised govt hospital in Aligarh."Many children are seriously injured," said senior police officer Daljeet Chaudhary. He added: "We will take action if anyone found at fault."Police say that according to a government order, all schools are shut till Friday because of cold weather conditions. An investigation has been ordered into why the school was functioning despite the government order.Home Minister Rajnath Singh tweeted: "My heart goes out to the families of those children who lost their lives in this tragedy. I pray for the speedy recovery of the injured." Deeply saddening to learn of the loss of many precious lives in a road accident in Etah district of Uttar Pradesh, this morning. - Rajnath Singh (@rajnathsingh) January 19, 2017 My heart goes out to the families of those children who lost their lives in this tragedy. I pray for the speedy recovery of the injured. - Rajnath Singh (@rajnathsingh) January 19, 2017 Police have started an investigation, both into the accident as well as to find out why the school was functioning, despite it being a holiday.. 
At least 15 children died and 30 were injured when a school bus collided with a truck Thursday in northern India, authorities said. 
"Some children are in critical condition, so casualties could rise," said Parashuram Singh, the deputy superintendent of police in the state of Uttar Pradesh's Etah district. He told CNN, the school bus was carrying around 60 junior high school students. 
An initial investigation indicates the truck was speeding in dense fog, which likely caused the accident, the deputy said. 
Etah district administration had ordered all schools in the district to remain closed until January 20 due to cold weather, Satish Pal, Additional District Magistrate (ADM) of the district told CNN. He said his department was making inquiries as to why this particular school remained open. 
In a tweet, Prime Minister Narendra Modi said "anguished by the tragic accident in UP's Etah district. I share the pain of the bereaved families & condole passing away of young children. I pray that those injured in the accident in Etah recover at the earliest." Lax safety 
India has a history of deadly road accidents. In 2015, the total number of traffic accidents increased 2.5 percent to 501,423, from 489,400 in 2014, according to the country's transport and highways ministry. Traffic-related deaths increased 4.6 percent to 146,133 in 2015, from 139,671 in 2014. 
Around 83.6 percent of traffic-related deaths occurred in India's top thirteen states during 2015. The highest number of deaths had taken place in Uttar Pradesh, which reported 17,666 fatalities. 
India sees an average of 1,374 traffic accidents a day -- as well as 400 deaths every day -- on roads. 
This equates to almost one accident per minute and one death every four minutes on roads -- making it a bigger killer than HIV/AIDS and other diseases, according to the World Health Organization. 
In a pair of particularly deadly incidents only two days apart last year in India, at least 29 people were killed -- including 17 children -- when two buses plunged into water. 
During a 2013 road safety campaign, the ministry said many of these incidents were due to drunk driving, a scourge it said needed to eliminated by spreading awareness and stringent enforcement measures. It added that state governments have been requested to remove liquor shops from the side of highways."""


text2 = """In a tragic incident, at least fifteen children have died after a school bus they were traveling in collided with a truck in Etah district of Uttar Pradesh. 
About 23 children have been injured in the tragic mishap. About 40 children were seated in the bus. 
The accident happened in Aliganj in Etah early in the morning, when the bus belonging to JS Public School collided with a truck. 
It was a head-on collision. The children in the bus were on their way to school. 
The school was open despite the Etah administration announcing three days of holidays till January 20 due to cold conditions. 
The school is likely to face action. 
Prime Minister Narendra Modi expressed his anguish over the tragic incident. 
While extending condolences to the bereaved families, PM Modi said: “Anguished by the tragic accident in UP’s Etah district. I share the pain of the bereaved families & condole passing away of young children.”"""

x = lang.encoder_pair(text1,text2)